# TTC6 Mutation Trans Effect on Transcriptomics

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

import cptac
import cptac.utils as al

brain = cptac.Gbm()

### Select Gene

In [2]:
gene = "TTC6"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

### Track all significant comparisons in Dataframe

In [4]:
all_significant_comparisons = pd.DataFrame(columns=['Cancer_Type', 'Gene', 'Comparison','Interacting_Protein','P_Value'])

In [5]:
def add_to_all_significant_comparisons(df, cancer, interacting, all_sig_comp):
    expanded = df
    expanded['Gene'] = gene
    expanded['Cancer_Type'] = cancer
    expanded['Interacting_Protein'] = interacting
    
    updated_all_comparisons = pd.concat([all_sig_comp, expanded], sort=False)
    
    return updated_all_comparisons

# Interacting Proteins: Transcriptomics

### Generate interacting protein list

Make a call to cptac.utils to get the interacting proteins method, which interacts with the uniprot and string databases to generate a list of known interacting partners with the given gene.

In [6]:
# Use get interacting proteins method to generate list of interacting proteins
interacting_proteins = al.get_interacting_proteins(gene)

# Show interacting protein list
print("Interacting Proteins:")
for interacting_protein in interacting_proteins:
    print(interacting_protein)


Specified gene was not found in String database, double check that you have it correctly!


TypeError: 'NoneType' object is not iterable

### Test for significant comparisons in any of interacting proteins

In [7]:
# Create dataframe in order to do comparisons with wrap_ttest
protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = brain.reduce_multiindex(protdf, flatten=True)
protdf = protdf[protdf.Sample_Status == 'Tumor'] # drop Normal samples

# Create the binary valued column needed to do the comparison
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

# Format the dataframe correctly
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


# Make list of columns to be compared using t-tests
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

# Call wrap_ttest, pass in formatted dataframe
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

# Print results, if anything significant was found
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", True, all_significant_comparisons)
        

NameError: name 'interacting_proteins' is not defined

# All Proteins: Transcriptomics 

In [8]:
try:
    print("\nGene: ", gene)

    # Use all proteins

    # Create dataframe in order to do comparisons with wrap_ttest
    protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = brain.reduce_multiindex(protdf, flatten=True) # flatten for specific column names
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    # Create binary column 
    protdf['Label'] = np.where(
                protdf[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

    # Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    # Make list of columns to be compared using t-tests
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    # Call wrap_ttest, pass in formatted dataframe
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    # Print results, if anything significant was found
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  TTC6


Doing t-test comparisons

                                          Comparison       P_Value
0       RP1-92C8.3_transcriptomics_ENSG00000226181.1  9.369644e-10
1       AC073869.7_transcriptomics_ENSG00000237910.1  1.183402e-09
2        LINC00692_transcriptomics_ENSG00000230891.5  1.183402e-09
3      Metazoa_SRP_transcriptomics_ENSG00000277591.1  1.524282e-09
4       RNU6-1059P_transcriptomics_ENSG00000200622.1  1.704726e-09
5        RPL23AP22_transcriptomics_ENSG00000237213.2  2.567268e-09
6       AC007251.2_transcriptomics_ENSG00000277527.1  7.332055e-09
7     RP11-363E6.1_transcriptomics_ENSG00000254014.1  1.943077e-08
8        LINC00383_transcriptomics_ENSG00000237534.1  2.437133e-08
9    CTD-2231E14.4_transcriptomics_ENSG00000267517.1  2.566330e-08
10        RN7SL42P_transcriptomics_ENSG00000263999.2  3.992894e-08
11   RP11-676B18.1_transcriptomics_ENSG00000279511.1  5.035850e-08
12          MAGEC2_transcriptomics_ENSG00000046774.9  7.374542e-08
13        KRT18P39_transcriptomics_E

# Print all significant comparisons

In [9]:
if len(all_significant_comparisons) > 0:
    display(all_significant_comparisons)
    
else:
    print('No Significant Comparisons!')

,Cancer_Type,Gene,Comparison,Interacting_Protein,P_Value
0,Gbm,TTC6,RP1-92C8.3_transcriptomics_ENSG00000226181.1,False,9.369644e-10
1,Gbm,TTC6,AC073869.7_transcriptomics_ENSG00000237910.1,False,1.183402e-09
2,Gbm,TTC6,LINC00692_transcriptomics_ENSG00000230891.5,False,1.183402e-09
3,Gbm,TTC6,Metazoa_SRP_transcriptomics_ENSG00000277591.1,False,1.524282e-09
4,Gbm,TTC6,RNU6-1059P_transcriptomics_ENSG00000200622.1,False,1.704726e-09
5,Gbm,TTC6,RPL23AP22_transcriptomics_ENSG00000237213.2,False,2.567268e-09
6,Gbm,TTC6,AC007251.2_transcriptomics_ENSG00000277527.1,False,7.332055e-09
7,Gbm,TTC6,RP11-363E6.1_transcriptomics_ENSG00000254014.1,False,1.943077e-08
8,Gbm,TTC6,LINC00383_transcriptomics_ENSG00000237534.1,False,2.437133e-08
9,Gbm,TTC6,CTD-2231E14.4_transcriptomics_ENSG00000267517.1,False,2.566330e-08


### Write Significant Comparisons (if any) to Shared CSV file

In [ ]:
'''
existing_results = pd.read_csv(gene+'_Trans_Results.csv')

updated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)

updated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)'''